In [3]:
# Install Dependencies
!pip install langchain langchain-community langchain-openai faiss-cpu tiktoken oracle-ads
!pip install sentence-transformers
!pip install transformers accelerate bitsandbytes sentence-transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.7/31.7 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75
ERROR: pip's dependenc

In [2]:
# Imports
import os
from google.colab import files

# LangChain imports
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline


In [4]:

# Upload File
uploaded = files.upload()
file_path = list(uploaded.keys())[0]
print("Uploaded file:", file_path)


Saving Etisalat solutions And Business Rules.txt to Etisalat solutions And Business Rules (1).txt
Uploaded file: Etisalat solutions And Business Rules (1).txt


In [9]:
# Load & Split Document
loader = TextLoader(file_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
print("docs",docs)


docs [Document(metadata={'source': 'Etisalat solutions And Business Rules (1).txt'}, page_content='Etisalat solutions for multiple projects\n\n1- Home move migration\n- If user is moving to any other emirate like from sharja to dubai they have to perform change plan as well. \n- api used for change plan is /homeMoveChangePlanJourney\n\n\nBusiness Ruls of Etisalat\n\n- As per current production, user cannot purchase smart device without postpaid plan\n- As per current production, user cannot be able to see NBA offers if he has only one account.')]


In [12]:
# Using a free model from HuggingFace
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build FAISS index with local embeddings
vectorstore = FAISS.from_documents(docs, embeddings)

/tmp/ipython-input-1651234082.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:

#model_name = "distilgpt2" best for colab test env
# =========================
# Load Local HuggingFace LLM
# =========================
# Using flan-t5-base (fast and instruction-tuned)
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
local_llm = HuggingFacePipeline(pipeline=pipe)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
/tmp/ipython-input-2116760608.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  local_llm = HuggingFacePipeline(pipeline=pipe)


In [27]:
# Retrieval QA
qa_chain = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

In [29]:
# Ask a Question
query = "what is the api used for home move migraiton with change plan?"
result = qa_chain({"query": query})
print("result:",result)

result: {'query': 'what is the api used for home move migraiton with change plan?', 'result': '/homeMoveChangePlanJourney Business Ruls of Etisalat ------------', 'source_documents': [Document(id='8575a77d-3397-4787-919b-ae680b2fcf87', metadata={'source': 'Etisalat solutions And Business Rules (1).txt'}, page_content='Etisalat solutions for multiple projects\n\n1- Home move migration\n- If user is moving to any other emirate like from sharja to dubai they have to perform change plan as well. \n- api used for change plan is /homeMoveChangePlanJourney\n\n\nBusiness Ruls of Etisalat\n\n- As per current production, user cannot purchase smart device without postpaid plan\n- As per current production, user cannot be able to see NBA offers if he has only one account.')]}


In [28]:
# Ask a Question
query = "Is it allowed to buy smart device without postpaid plan?"
result = qa_chain({"query": query})
print("result:",result)

result: {'query': 'Is it allowed to buy smart device without postpaid plan?', 'result': 'user cannot purchase smart device without postpaid plan ------------', 'source_documents': [Document(id='8575a77d-3397-4787-919b-ae680b2fcf87', metadata={'source': 'Etisalat solutions And Business Rules (1).txt'}, page_content='Etisalat solutions for multiple projects\n\n1- Home move migration\n- If user is moving to any other emirate like from sharja to dubai they have to perform change plan as well. \n- api used for change plan is /homeMoveChangePlanJourney\n\n\nBusiness Ruls of Etisalat\n\n- As per current production, user cannot purchase smart device without postpaid plan\n- As per current production, user cannot be able to see NBA offers if he has only one account.')]}
